In [ ]:
import numpy as np
import pandas as pd
from keras.utils import np_utils
np.random.seed(10)
from keras.datasets import mnist
(X_train_image, y_train_label),\
(X_test_image, y_test_label) = mnist.load_data()
x_Train = X_train_image.reshape(60000,784).astype('float32')
x_Test = X_test_image.reshape(10000,784).astype('float32')
x_Train_normalized = x_Train/225
x_Test_normalized = x_Test/225
y_TrainOneHot = np_utils.to_categorical(y_train_label)
y_TestOneHot = np_utils.to_categorical(y_test_label)

In [ ]:
import matplotlib.pyplot as plt
def plot_image(image):
    fig = plt.gcf()
    fig.set_size_inches(5,5)
    plt.imshow(image,cmap='binary')
    plt.show()

In [ ]:
import numpy as np
import copy
import math
class CNN:
    def __init__(self,layers,Clayers):
        self.layers = layers
        self.convolution = 1
        self.nums = len(layers)
        self.result = []
        self.weight = [np.random.randn(x,y) for x,y in zip(layers[1:],layers)]
        self.bias = [np.random.randn(p) for p in layers[1:]]
        self.loss = []
        self.data_a = [np.ones((x)) for x in layers]
        self.data_z = [np.ones((x)) for x in layers[1:]]
        self.kernel = [np.random.randn(x,y,3,3) for x,y in zip(Clayers[1:],Clayers)]
        self.convol_bias = [np.zeros((x)) for x in Clayers[1:]]
        self.data_convol = []
        self.data_convol_z = []
        self.data_pooling = []
    def feedforward(self):
        self.data_a[0] = np.array(self.data_pooling[len(self.data_pooling)-1])
        temp_reform = (self.data_a[0].shape)
        self.data_a[0] = self.data_a[0].reshape(temp_reform[0]*temp_reform[1]*temp_reform[2])
        for i in range (self.nums-1): 
            z = np.matmul(self.weight[i],self.data_a[i])
            z += self.bias[i]
            temp = copy.copy(z)
            self.data_z[i] = temp
            #z = ReLU_FC(z)
            z = sigmoid(z)
            self.data_a[i+1] = z
        self.result = self.data_a[self.nums-1]
    def backpropagation(self,correction,learning_rate):
        C0_deri = (self.result-correction)*2
        C0_deri = np.array(C0_deri)
        self.loss.append(np.sum(np.square(self.result-correction)))
        for i in range(self.nums-2,-1,-1):
            #sigmoid = np.array(ReLU_deri_FC(self.data_z[i]),dtype='float64')
            sigmoid = np.array(sigmoid_der(self.data_z[i]),dtype='float64')
            sigmoid = sigmoid*C0_deri
            temp = np.array([sigmoid])
            C0_deri = np.matmul(self.weight[i].T,sigmoid)
            weight = np.matmul(np.transpose(temp),np.array([self.data_a[i]]))
            self.learning(i,np.array(weight),np.array(sigmoid),learning_rate,correction)
        self.backprop_nonFC(C0_deri.reshape(12,13,13),self.convolution-1,learning_rate)
    def convolutionLayer(self,input_):
        self.data_pooling.append([input_])
        for i in range(self.convolution):
            output = np.zeros((int(len(self.kernel[i])),len(self.data_pooling[i][0])-2,len(self.data_pooling[i][0])-2))
            for j in range(len(self.kernel[i])):
                sum = np.zeros((len(self.data_pooling[i][0])-2,len(self.data_pooling[i][0])-2))
                for k in range(len(self.kernel[i][0])):
                    sum+=convolute(self.data_pooling[i][k],self.kernel[i][j][k])
                output[j]=sum+self.convol_bias[i][j]
            self.data_convol_z.append(copy.copy(output))
            temp = sigmoid(output)
            #temp = ReLU(output)
            self.data_pooling.append(average_pooling(temp))
    def backprop_nonFC(self,input_deri,position,learning_rate):
        if(position<0):
            self.data_convol.clear()
            self.data_convol_z.clear()
            self.data_pooling.clear()
            return 0
        C0_deri = []
        for j in range (len(input_deri)):
            C0_deri.append(backprop_pooling(input_deri[j]))
        #C0_F = C0_deri*ReLU_deri(self.data_convol_z[position])
        C0_F = C0_deri*sigmoid_der(self.data_convol_z[position])
        bias_deri = []
        for i in range(len(C0_F)):
            bias_deri.append(np.sum(C0_F[i]))
        self.learningBias(position,bias_deri,learning_rate)
        for i in range(len(C0_F)):
            for j in range(len(self.data_pooling[position])):
                C0_k = convolute(self.data_pooling[position][j],C0_F[i])
                self.learningConvol(position,i,j,C0_k,learning_rate)
        C0_next=[]
        for i in range(len(self.data_pooling[position])):
            sum = np.zeros((len(self.data_pooling[position][0]),len(self.data_pooling[position][0])))
            for j in range(len(self.kernel[position])):
                sum+=full_convolute(self.kernel[position][j][i],C0_F[j])
            C0_next.append(sum)
        self.backprop_nonFC(C0_next,position-1,learning_rate)
    def learning(self,i,weight,bias,learning_rate,correction):
        n = -1*learning_rate
        delta_bias = n*bias
        delta_weight = n*weight
        self.weight[i]+=delta_weight
        self.bias[i]+=delta_bias
    def learningConvol(self,i,j,k,kernel,learning_rate):
        print(kernel)
        n = -1*learning_rate
        delta_kernel = n*kernel
        self.kernel[i][j][k] += delta_kernel
    def learningBias(self,i,bias_deri,learning_rate):
        n = -1*learning_rate
        delta_kernel = np.array(bias_deri)
        delta_kernel = delta_kernel*n
        self.convol_bias[i]+=delta_kernel
def average_pooling(a):
    output = []
    for i in range(len(a)):
        s = np.zeros((int(len(a[i])/2),int(len(a[i])/2)))
        for j in range(len(a[i])):
            for k in range(len(a[i])):
                s[int(j/2),int(k/2)]+=a[i][j][k]/4
        output.append(s)
    return output
def backprop_pooling(C0):
    output = np.zeros((len(C0)*2,len(C0)*2))
    for i in range(len(C0)*2):
        for j in range(len(C0)*2):
            output[i][j] = C0[int(i/2)][int(j/2)]*1/4
    return output
def convolute(a,b):
    output = []
    for i in range(len(a)-len(b)+1):
        temp = []
        for j in range(len(a[0])-len(b)+1):
            sum_ = 0
            for k in range(len(b)):                    
                sum_ += np.dot(a[i+k][j:j+len(b[k])],b[k])
            temp.append(sum_)
        output.append(temp)
    return np.array(output)
def full_convolute(a,b):
    rotate = np.zeros((len(b),len(b)))
    for i in range(len(b)):
        rotate[i][len(b)-i-1] = 1
    b = np.matmul(rotate,b)
    wrap = np.pad(a, len(b)-1, pad_with)
    return convolute(wrap,b)
def sigmoid(z):
    return 1.0/(1.0+np.exp(-1*z))
def sigmoid_der(z):
    a = sigmoid(z)
    return a*(1-a)
def ReLU(z):
    for k in range(len(z)):
        for i in range(len(z[0])):
            for j in range(len(z[0][0])):
                if(z[k][i][j]<0):
                    z[k][i][j]=max(0,z[k][i][j])
    return z
def ReLU_deri(z):
    output = np.zeros((len(z),len(z[0]),len(z[0][0])))
    for k in range(len(z)):
        for i in range(len(z[0])):
            for j in range(len(z[0][0])):
                if(z[k][i][j]>=0):
                    output[k][i][j] = 1
                else:
                    output[k][i][j] = 0
    return output
def ReLU_FC(z):
    c = np.zeros((len(z)))
    for i in range(len(z)):
        if(z[i]<0):
            c[i]=z[i]*0.01
        else:
            c[i]=z[i]
    return c
def ReLU_deri_FC(z):
    c = np.zeros((len(z)))
    for i in range(len(z)):
        if(z[i]>0):
            c[i] = 1
        else:
            c[i] = 0.01
    return c
def pad_with(vector, pad_width, iaxis, kwargs):
    pad_value = kwargs.get('padder', 0)
    vector[:pad_width[0]] = pad_value
    vector[-pad_width[1]:] = pad_value
network = CNN([2028,400,10],[1,12])

In [ ]:
import matplotlib.pyplot as plt
a = []
b = []
network.loss.clear()
for p in range(30):
    b.append(p)
    sum_value = 0
    for i in range (100):
        network.convolutionLayer(x_Train_normalized[100*p+i].reshape(28,28))
        network.feedforward()
        index = np.argmax(network.result)
        if(index==y_train_label[100*p+i]):
            sum_value+=1     
        network.backpropagation(y_TrainOneHot[100*p+i],0.25)
    print("epoch ",p+1)
    print(sum_value,"/100")
    print("accuracy",sum_value,"%")
    a.append(np.sum(network.loss))
    network.loss.clear()
    sum_value=0
    for i in range(100):
        network.convolutionLayer(x_Test_normalized[i].reshape(28,28))
        network.feedforward()
        index = np.argmax(network.result)
        if(index==y_test_label[i]):
            sum_value+=1   
    print(sum_value,"/100")
    print("accuracy",sum_value,"%")
plt.plot(b,a)